In [71]:
#imports

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras import layers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.models import Sequential

from PIL import Image
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report

In [72]:
#connect google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
#data directories

import pathlib
data_dir = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray"
data_dir_train = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/train"
data_dir_test = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/test"
data_dir_validation = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/val"

In [74]:
#check and pull images

from pathlib import Path
import imghdr

data_dir = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray"
image_extensions = [".png", ".svg", ".gif", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [75]:
# find bad file????

from pathlib import Path
import imghdr

img_link=list(Path("/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)
#this should help you delete the bad encoded

Total 0 pcs image delete from Dataset


In [76]:


img_height, img_width = 64, 64
batch_size = 32

#process datasets

#batch size
#image size

#train
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

#test
test_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_test,
                                                              image_size =(img_height, img_width),
                                                              batch_size=batch_size)

#validation
val_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_validation,
                                                             image_size=(img_height, img_width),
                                                             batch_size=batch_size)



#Creating three datases of images from a directory of image files. This function returns a tf.data.Dataset containing batches of images and their corresponding labels.
#it will contain image and label pairs that can be used to train a neural network.

Found 4729 files belonging to 2 classes.
Found 586 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [77]:
#class names
class_names = train_ds.class_names
print(class_names)

num_classes = 2

['NORMAL', 'PNEUMONIA']


# Modeling

In [78]:
#imports

from keras import models
from keras import layers
from keras import optimizers

In [70]:
#This script essentially trains a neural network model on the training dataset,
# periodically logs training progress, and evaluates the model's performance on a separate test dataset after training.

iterator = iter(train_ds)
max_iterations = len(train_ds)
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    i = 0
    while i < max_iterations:
        print("Currently running {} batch".format(i))
        try:
            i = i + 1
            x_batch_train, y_batch_train = next(iterator)
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

            # Log every 200 batches.
            if i % 200 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (i, float(loss_value))
                )
                print("Seen so far: %s samples" % ((i + 1) * batch_size))

            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

            # Reset training metrics at the end of each epoch
            train_acc_metric.reset_states()
            for x_batch_val, y_batch_val in val_ds:
                val_logits = model(x_batch_val, training=False)
                # Update val metrics
                val_acc_metric.update_state(y_batch_val, val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
        except Exception as e:
            continue

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)


Start of epoch 0
Currently running 0 batch
Currently running 1 batch
Currently running 2 batch
Currently running 3 batch
Currently running 4 batch
Currently running 5 batch
Currently running 6 batch
Currently running 7 batch
Currently running 8 batch
Currently running 9 batch
Currently running 10 batch
Currently running 11 batch
Currently running 12 batch
Currently running 13 batch
Currently running 14 batch
Currently running 15 batch
Currently running 16 batch
Currently running 17 batch
Currently running 18 batch
Currently running 19 batch
Currently running 20 batch
Currently running 21 batch
Currently running 22 batch
Currently running 23 batch
Currently running 24 batch
Currently running 25 batch
Currently running 26 batch
Currently running 27 batch
Currently running 28 batch
Currently running 29 batch
Currently running 30 batch
Currently running 31 batch
Currently running 32 batch
Currently running 33 batch
Currently running 34 batch
Currently running 35 batch
Currently running 36

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


19/19 [==============================] - 7s 246ms/step - loss: 0.7519 - accuracy: 0.3993


In [35]:
#first model attempt

fsm = models.Sequential()
fsm.add(layers.Dense(10, activation='sigmoid', input_shape=(64, 64, 3)))
fsm.add(layers.Flatten())
fsm.add(layers.Dense(2, activation='sigmoid'))

In [36]:
#compile
fsm.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
#run on fsm

epochs = 3
# history = fsm.fit(train_ds, validation_data = val_ds, epochs = epochs) ## does not run

Epoch 1/3


InvalidArgumentError: ignored